<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/remoci%C3%B3n_de_fondo/alpha_matting_con_rembg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción

Este notebook utiliza la técnica *alpha matting* de la librería *rembg* para extraer un objeto de primer plano con límites suaves de una imagen de fondo, es decir, remover el fondo de una imagen dada.

El mateado de imágenes es el proceso de extraer un mate alfa preciso que separa los objetos de primer plano y de fondo en una imagen. Esta técnica se ha utilizado tradicionalmente en la industria cinematográfica y fotográfica para fines de edición de imágenes y videos, por ejemplo, reemplazo de fondo, efecto bokeh sintético y otros efectos visuales. La superposición de imágenes supone que una imagen es una combinación de imágenes de primer plano y de fondo y, por lo tanto, la intensidad de cada píxel es una combinación lineal del primer plano y el fondo.

En el caso de la segmentación tradicional de imágenes, la imagen se segmenta de forma binaria, en la que un píxel pertenece al primer plano o al fondo. Este tipo de segmentación, sin embargo, no puede tratar con escenas naturales que contienen detalles finos, por ejemplo, cabello y pelaje, que requieren estimar un valor de transparencia para cada píxel del objeto en primer plano.

In [ ]:
!pip install rembg gradio

In [2]:
import os
import cv2
import gradio as gr

from rembg import remove
from rembg.session_base import BaseSession
from rembg.session_factory import new_session

import warnings
warnings.filterwarnings("ignore")

In [3]:
def inference(file, af, mask, model):
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    cv2.imwrite(os.path.join("input.png"), img)

    input_path = 'input.png'
    output_path = 'output.png'

    with open(input_path, 'rb') as i:
        with open(output_path, 'wb') as o:
            input = i.read()
            sessions: dict[str, BaseSession] = {}
            output = remove(input, session=sessions.setdefault(model, new_session(model)),
                            alpha_matting_erode_size = af,
                            only_mask = (True if mask == "Mask only" else False))
            o.write(output)
    return os.path.join("output.png")

In [4]:
title = "Remover Fondo"
description = "Demostración de la libraría rembg. Para usarlo, simplemente cargue una imagen, seleccione un modelo y envíe."

In [5]:
gr.Interface(inference,
    [gr.inputs.Image(type="filepath", label="Imagen de entrada"),
     gr.inputs.Slider(10, 25, default=10, label="Ajuste de 'alpha matting'"), 
     gr.inputs.Radio(["Default", "Mask only"], type="value", default="Default", label="Opciones"),
     gr.inputs.Dropdown(["u2net", "u2netp", "u2net_human_seg", "u2net_cloth_seg", "silueta"],
                        type="value", default="u2net", label="Modelos")], 
    gr.outputs.Image(type="pil", label="Imagen de salida"),
    title=title, description=description, enable_queue=True).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://009e422836b63f3106.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


**Referencias**

* https://gradio.app/docs/#image
* https://github.com/danielgatis/rembg
* https://docs.opencv.org/4.x/d4/d40/group__alphamat.html